In [1]:
# %load /home/hwixley/Documents/Year3/IAML/cwk2/INFR10069-2020-CW2/helpers/iaml01cw2_helpers.py
#
#  Helper functions for IAML 2020/21 cw2
#
import os
import gzip
import numpy as np

#
#  This function is after  https://github.com/zalandoresearch/fashion-mnist
#
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

#
#------ IAML original functions ---------------
#
def load_FashionMNIST(data_path):
    Xtrn, Ytrn = load_mnist(data_path, "train")
    Xtst, Ytst = load_mnist(data_path, "t10k")
    return Xtrn.astype(np.float), Ytrn, Xtst.astype(np.float), Ytst

def load_CoVoST2(data_path):
    data = np.load(data_path+'/speech_trn.npz', allow_pickle=True)
    Xtrn, Ytrn = data['Xtrn'], data['Ytrn']
    data = np.load(data_path+'/speech_tst.npz', allow_pickle=True)
    Xtst, Ytst = data['Xtst'], data['Ytst']
    return Xtrn, Ytrn, Xtst, Ytst


In [19]:

##########################################################
#  Python script template for Question 1 (IAML Level 10)
#  Note that
#  - You should not change the filename of this file, 'iaml01cw2_q1.py', which is the file name you should use when you submit your code for this question.
#  - You should define the functions shown below in your code.
#  - You can define function arguments (parameters) and returns (attributes) if necessary.
#  - In case you define helper functions, do not define them here, but put them in a separate Python module file, "iaml01cw2_my_helpers.py", and import it in this script.
#  - For those questions requiring you to show results in tables, your code does not need to present them in tables - just showing them with print() is fine.
#  - You do not need to include this header in your submission
##########################################################

#--- Code for loading the data set and pre-processing --->
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data:
Xtrn, Ytrn, Xtst, Ytst = load_FashionMNIST("/home/hwixley/Documents/Year3/IAML/cwk2/INFR10069-2020-CW2/data")
Xtrn_orig = Xtrn.copy
Xtst_orig = Xtst.copy
Xtrn = Xtrn/255
Xtst = Xtst/255

Xmean = Xtrn.mean(0)
Xtrn_nm = Xtrn - Xmean
Xtst_nm = Xtst - Xmean
#<----

# Q1.1
def iaml01cw2_q1_1():
    print(Xtrn_nm[0,0:4])
    print(Xtrn_nm[-1,0:4])
#
# iaml01cw2_q1_1()   # comment this out when you run the function

# Q1.2
#def iaml01cw2_q1_2():
#
# iaml01cw2_q1_2()   # comment this out when you run the function

# Q1.3
#def iaml01cw2_q1_3():
#
# iaml01cw2_q1_3()   # comment this out when you run the function


# Q1.4
#def iaml01cw2_q1_4():
#
# iaml01cw2_q1_4()   # comment this out when you run the function


# Q1.5
#def iaml01cw2_q1_5():
#
# iaml01cw2_q1_5()   # comment this out when you run the function


# Q1.6
#def iaml01cw2_q1_6():
#
# iaml01cw2_q1_6()   # comment this out when you run the function


# Q1.7
#def iaml01cw2_q1_7():
#
# iaml01cw2_q1_7()   # comment this out when you run the function


# Q1.8
#def iaml01cw2_q1_8():
#
# iaml01cw2_q1_8()   # comment this out when you run the function


[-3.13725490e-06 -2.26797386e-05 -1.17973856e-04 -4.07058824e-04]
[-3.13725490e-06 -2.26797386e-05 -1.17973856e-04 -4.07058824e-04]
